In [ ]:
username       = <ENTER_USERNAME>
password       = <ENTER_PASSWORD>
profile_type   = 'okr-sprout'

In [ ]:
import requests
import urllib

url = 'https://demo.grnry.io/auth/realms/grnry/protocol/openid-connect/token'

payload = f'grant_type=password&username={urllib.parse.quote_plus(username)}&password={urllib.parse.quote_plus(password)}&client_id=profile-api'
headers = {
  'Content-Type': 'application/x-www-form-urlencoded'
}

response = requests.request('POST', url, headers=headers, data = payload)

token = response.json()['access_token']

In [ ]:
url = f'https://demo.grnry.io/profiles/{profile_type}'

payload={}
headers = {
  'Authorization': f'Bearer {token}'
}

response = requests.request('GET', url, headers=headers, data=payload).json()

In [ ]:
import pandas as pd

profiles = [ {'circle':p['correlationId'],**p['jsonPayload']} for p in response['profiles']]
circles = pd.json_normalize(profiles,max_level=2)
circles = circles.melt(id_vars=['circle'])
values = pd.json_normalize(circles['value'],max_level=0)
values = values.applymap(lambda x: x.get('_latest').get('_v') if isinstance(x, dict) else '0')
okr_progress = pd.concat([circles[['circle','variable']],values],axis=1)
okr_progress

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

plt.rcdefaults()
fig, ax = plt.subplots()

circle = okr_progress['circle']
y_pos = np.arange(len(circle))
performance = (okr_progress['numberOfKR'].astype(int)*100/
               okr_progress['progressOfKR'].astype(int)).tolist()
error = np.random.rand(len(circle))

ax.barh(y_pos, performance, xerr=error, align='center')
ax.set_yticks(y_pos)
ax.set_yticklabels(circle)
ax.invert_yaxis()  # labels read top-to-bottom
ax.set_xlabel('Progress')
ax.set_title('OKR Progress per circle in percent')

plt.show()